GeekBrains, факультет Искусственного интелекта <br>
курс "Введение в обработку естественного языка" (старт 21 июня 2023)

### Урок 2. Создание признакового пространства

Домашнее задание

### Задание 1.

Задание: обучите три классификатора:

1) на токенах с высокой частотой

2) на токенах со средней частотой

3) на токенах с низкой частотой

Сравните полученные результаты, оцените какие токены наиболее важные для классификации.

### Задание 2.
найти фичи с наибольшей значимостью, и вывести их

### Задание 3.
1) сравнить count/tf-idf/hashing векторайзеры/полносвязанную сетку (построить classification_report)

2) подобрать оптимальный размер для hashing векторайзера

3) убедиться что для сетки нет переобучения

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import *
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

from nltk.tokenize import word_tokenize
from string import punctuation

from collections import Counter

from sklearn.linear_model import LogisticRegression # можно заменить на любимый классификатор
from sklearn.feature_extraction.text import CountVectorizer
import warnings
warnings.filterwarnings("ignore")

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [2]:
# считываем данные и заполняем общий датасет
positive = pd.read_csv('data/positive.csv', sep=';', usecols=[3], names=['text'])
positive['label'] = 1
negative = pd.read_csv('data/negative.csv', sep=';', usecols=[3], names=['text'])
negative['label'] = 0
df = positive.append(negative)

In [3]:
df.head()

,text,label
0,"@first_timee хоть я и школота, но поверь, у на...",1
1,"Да, все-таки он немного похож на него. Но мой ...",1
2,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,1
3,"RT @digger2912: ""Кто то в углу сидит и погибае...",1
4,@irina_dyshkant Вот что значит страшилка :D\nН...,1


In [4]:
df.tail()

,text,label
111918,Но не каждый хочет что то исправлять:( http://...,0
111919,скучаю так :-( только @taaannyaaa вправляет мо...,0
111920,"Вот и в школу, в говно это идти уже надо(",0
111921,"RT @_Them__: @LisaBeroud Тауриэль, не грусти :...",0
111922,Такси везет меня на работу. Раздумываю приплат...,0


In [5]:
x_train, x_test, y_train, y_test = train_test_split(df.text, df.label)

#### Посмотрим на качество с полным корпусом слов.

In [6]:
vec = CountVectorizer(ngram_range=(1, 1))
bow = vec.fit_transform(x_train) # bow -- bag of words (мешок слов)

In [7]:
clf = LogisticRegression(random_state=42)
clf.fit(bow, y_train)

LogisticRegression(random_state=42)

In [8]:
pred = clf.predict(vec.transform(x_test))
print(classification_report(pred, y_test))

              precision    recall  f1-score   support

           0       0.77      0.76      0.76     28185
           1       0.76      0.77      0.77     28524

    accuracy                           0.77     56709
   macro avg       0.77      0.77      0.77     56709
weighted avg       0.77      0.77      0.77     56709



#### Соберем корпус из всех слов

In [9]:
corpus = [token for tweet in df.text for token in word_tokenize(tweet) if token not in punctuation]
print(len(corpus))

2870536


In [10]:
freq_dict = Counter(corpus)
freq_dict_sorted = sorted(freq_dict.items(), key=lambda x: -x[1])

#### Сортируем уникальные слова по их частоте

In [11]:
len(freq_dict_sorted)  # Всего уникальных слов

351101

#### Возьмем первые 10к популярных слов

In [12]:
first_try = freq_dict_sorted[10000:]

In [13]:
first_try = [x[0] for x in first_try]

In [14]:
vec = CountVectorizer(ngram_range=(1, 1), stop_words=first_try)
bow = vec.fit_transform(x_train) # bow -- bag of words (мешок слов)

In [15]:
clf = LogisticRegression(random_state=42)
clf.fit(bow, y_train)

LogisticRegression(random_state=42)

In [16]:
pred = clf.predict(vec.transform(x_test))
print(classification_report(pred, y_test))

              precision    recall  f1-score   support

           0       0.74      0.74      0.74     27781
           1       0.75      0.75      0.75     28928

    accuracy                           0.74     56709
   macro avg       0.74      0.74      0.74     56709
weighted avg       0.74      0.74      0.74     56709



#### Возьмем 160к слов из середины частотного списка

In [17]:
second_try = freq_dict_sorted[:10000] + freq_dict_sorted[170000:]
second_try = [x[0] for x in second_try]

In [18]:
vec = CountVectorizer(ngram_range=(1, 1), stop_words=second_try)
bow = vec.fit_transform(x_train) # bow -- bag of words (мешок слов)
clf = LogisticRegression(random_state=42)
clf.fit(bow, y_train)

LogisticRegression(random_state=42)

In [19]:
pred = clf.predict(vec.transform(x_test))
print(classification_report(pred, y_test))

              precision    recall  f1-score   support

           0       0.75      0.63      0.69     33090
           1       0.58      0.70      0.63     23619

    accuracy                           0.66     56709
   macro avg       0.66      0.67      0.66     56709
weighted avg       0.68      0.66      0.66     56709



#### Возьмем последние 50к слов из частотного списка

In [20]:
last_try = freq_dict_sorted[:300000]
last_try = [x[0] for x in last_try]

In [21]:
vec = CountVectorizer(ngram_range=(1, 1), stop_words=last_try)
bow = vec.fit_transform(x_train) # bow -- bag of words (мешок слов)
clf = LogisticRegression(random_state=42)
clf.fit(bow, y_train)

LogisticRegression(random_state=42)

In [22]:
pred = clf.predict(vec.transform(x_test))
print(classification_report(pred, y_test))

              precision    recall  f1-score   support

           0       0.93      0.53      0.68     48699
           1       0.21      0.75      0.33      8010

    accuracy                           0.56     56709
   macro avg       0.57      0.64      0.50     56709
weighted avg       0.83      0.56      0.63     56709



### Из 350000 уникальных слов взял первые 10000, что показало лучший результат.  Т.е. самые популярные - самые важные.

In [23]:
vec = CountVectorizer(ngram_range=(1, 1))
bow = vec.fit_transform(x_train) # bow -- bag of words (мешок слов)

In [24]:
clf = LogisticRegression(random_state=42)
clf.fit(bow, y_train)

LogisticRegression(random_state=42)

In [25]:
pred = clf.predict(vec.transform(x_test))
print(classification_report(pred, y_test))

              precision    recall  f1-score   support

           0       0.77      0.76      0.76     28185
           1       0.76      0.77      0.77     28524

    accuracy                           0.77     56709
   macro avg       0.77      0.77      0.77     56709
weighted avg       0.77      0.77      0.77     56709



In [26]:
list_feat_names = [x[0] for x in list(vec.vocabulary_.items())]

In [27]:
features = [x for x in zip(clf.coef_[0], list_feat_names)]

In [28]:
sorted_features = sorted(features, key=lambda x: -abs(x[0]))

### Отсортированные фичи по весам (по модулю)

In [29]:
sorted_features[:50]

[(-8.38809400584574, 'отмороженный'),
 (-6.929360394904922, 'огурчиком'),
 (-6.891364923368163, 'ясное'),
 (-6.089948465829983, 'пробуждение'),
 (-5.779636107878169, 'строптивых'),
 (5.615610471507208, 'нино'),
 (5.600025608970472, 'переживание'),
 (-5.156976886229185, '22года'),
 (-5.054725833032386, 'гроффом'),
 (4.983344093613355, 'медаль'),
 (-4.960216604281107, 'қазнеттің'),
 (-4.599998908571897, 'человеком'),
 (4.516170752008647, 'serialofil'),
 (-4.419244935328321, 'чота'),
 (-4.318291814166679, 'самооценка'),
 (-4.2626387874812135, 'grrdfrnk'),
 (-4.169396683222514, 'oynbbgid0r'),
 (-3.9334570222309333, 'djlina1d'),
 (-3.592742882354733, 'распёрло'),
 (-3.588186281034321, 'сердюкова'),
 (-3.511717447186388, 'татарин'),
 (3.5010915654338186, 'ишга'),
 (3.436869700461141, 'погибли'),
 (-3.424389108327987, 'doninagalina'),
 (-3.385506230110895, '4001'),
 (-3.376380714785877, 'ограниченные'),
 (-3.3533515750347127, 'нова'),
 (-3.157716728953643, 'freedomchao'),
 (-3.096035969464435

### Потестим разные эмбединги

#### CountV

In [30]:
vec = CountVectorizer(ngram_range=(1, 1))
bow = vec.fit_transform(x_train) # bow -- bag of words (мешок слов)

In [31]:
clf = LogisticRegression(random_state=42)
clf.fit(bow, y_train)

LogisticRegression(random_state=42)

In [32]:
pred = clf.predict(vec.transform(x_train))
print(classification_report(pred, y_train))

              precision    recall  f1-score   support

           0       0.91      0.91      0.91     84147
           1       0.91      0.91      0.91     85978

    accuracy                           0.91    170125
   macro avg       0.91      0.91      0.91    170125
weighted avg       0.91      0.91      0.91    170125



In [33]:
clf = LogisticRegression(random_state=42)
clf.fit(bow, y_train)

LogisticRegression(random_state=42)

In [34]:
pred = clf.predict(vec.transform(x_test))
print(classification_report(pred, y_test))

              precision    recall  f1-score   support

           0       0.77      0.76      0.76     28185
           1       0.76      0.77      0.77     28524

    accuracy                           0.77     56709
   macro avg       0.77      0.77      0.77     56709
weighted avg       0.77      0.77      0.77     56709



#### Tf-Idf

In [35]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [36]:
vec = TfidfVectorizer(ngram_range=(1, 1))
bow = vec.fit_transform(x_train) # bow -- bag of words (мешок слов)

In [37]:
clf = LogisticRegression(random_state=42)
clf.fit(bow, y_train)

LogisticRegression(random_state=42)

In [38]:
pred = clf.predict(vec.transform(x_train))
print(classification_report(pred, y_train))

              precision    recall  f1-score   support

           0       0.83      0.86      0.84     80912
           1       0.87      0.84      0.85     89213

    accuracy                           0.85    170125
   macro avg       0.85      0.85      0.85    170125
weighted avg       0.85      0.85      0.85    170125



In [39]:
pred = clf.predict(vec.transform(x_test))
print(classification_report(pred, y_test))

              precision    recall  f1-score   support

           0       0.74      0.77      0.75     26743
           1       0.79      0.76      0.77     29966

    accuracy                           0.76     56709
   macro avg       0.76      0.76      0.76     56709
weighted avg       0.76      0.76      0.76     56709



#### Hashing
*подобрал оптимально кеоличество фичей - 10к, при увеличении в дальнейшем прирост качества не значительный, относительно количества увеличенных фичей.

In [40]:
from sklearn.feature_extraction.text import HashingVectorizer

In [41]:
vec = HashingVectorizer(n_features=10000)
bow = vec.fit_transform(x_train) # bow -- bag of words (мешок слов)

In [42]:
clf = LogisticRegression(random_state=42)
clf.fit(bow, y_train)

LogisticRegression(random_state=42)

In [43]:
pred = clf.predict(vec.transform(x_train))
print(classification_report(pred, y_train))

              precision    recall  f1-score   support

           0       0.73      0.75      0.74     81879
           1       0.76      0.74      0.75     88246

    accuracy                           0.74    170125
   macro avg       0.74      0.74      0.74    170125
weighted avg       0.74      0.74      0.74    170125



In [44]:
clf = LogisticRegression(random_state=42)
clf.fit(bow, y_train)

LogisticRegression(random_state=42)

In [45]:
pred = clf.predict(vec.transform(x_test))
print(classification_report(pred, y_test))

              precision    recall  f1-score   support

           0       0.70      0.71      0.71     27370
           1       0.73      0.71      0.72     29339

    accuracy                           0.71     56709
   macro avg       0.71      0.71      0.71     56709
weighted avg       0.71      0.71      0.71     56709



#### TF-IDF показал себя лучше остальных, однако Hashing позволил сократить размерность, что сильно повлияло на скорость. Также из всех моделей при использовании Hasing обобщающая способность модели лучше всего, модель почти не переобучилась. 